# EKF - Final Version

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
import random

## Simulation of a sensor's measurements ##

In this code, I try to implement an Extended Kalman Filter for vessel localisation. Specifically, I want to have an estimate of the boat's position, orientation and speed as the final output.

- In the 'Data' section, data is collected from the sensors.

- In the 'Implementation' section, the actual EKF is implemented. The function needs the sensor measurements and the number of iterations performed so far as input. The function gives 5 outputs in the following order: speed Vx, speed Vy, position x, position y, heading h, covariance matrix, kalman filter gain matrix, residual and n° of iterations. The last four outputs are completely useless for a pratical purpose but are essential to understand if the function is working properly.

- In the "Test" section, the function ekfilter() is called in order to test the function.

- In the "Plotting" section, some significant variables are plotted.

##### _N.B. as input, the function requires an array and an integer._ 

### Data

In [ ]:
def getMeasurement():
    Longitude = []
    Latitude = []
    Azimuth = []
    CovMatrix = []
    accelerazione = []
    Initial_lon = 7.416667 # Initialize lon value
    Initial_lat = 43.733334 # Initialize lat value
    Initial_az = 3.14 # Initialize lat value

    for iii in range(0,300):
        w = 0.001 * random.uniform(-1, 1) # random noise for lon
        v = 0.001 * random.uniform(-1, 1) # random noise for lat
        z = 0.001 * random.uniform(-1, 1) # random noise for az
        
        lon = Initial_lon + 0.05*iii  + w  # evolution of the longitude
        lat = Initial_lat + 0.05*iii  + v # evolotion of the latitude
        az = Initial_az + 0.15*iii + z # Linear evolotion of the Azimuth (degrees)
        a = 0.1*iii + 0.1*random.uniform(-1, 1) # Linear evolution of the acceleration (m/s^2)
        R = 1

        Longitude.append(lon)
        Latitude.append(lat)
        Azimuth.append(az)
        CovMatrix.append(R)
        accelerazione.append(a)
    
    return [Longitude, Latitude, Azimuth, CovMatrix, accelerazione]


#### Implementation

In [ ]:
def ekfilter(z, updateNumber):
    j = updateNumber
    Radius = 6371*10**3 # Earth radius in meters
    phi1 = 43.733334 # average between lower and upper bound of the latitude of the map you want to consider
    dt = 1.0 # Sampling time

    # Initialize State
    if updateNumber == 0: # First Update

        # compute position values from measurements
        # x = R*lon*cos(phi0)
        temp_x = Radius*z[0][j]*np.cos(phi1)
        # y = R*lat
        temp_y = Radius*z[1][j]
        # azimuth = azimuth
        temp_h = z[2][j]

        # state vector
        # - initialize position values
        ekfilter.x = np.array([[temp_x],
                               [temp_y],
                               [0],
                               [0],
                               [temp_h]])

        # state covariance matrix
        # - initialized to zero for first update
        ekfilter.P = np.array([[0, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0]])

        # state matrix A
        # - linear extrapolation
        ekfilter.A = np.array([[dt*np.cos(temp_h), dt*np.sin(temp_h), 1, 0, -dt*temp_x*np.sin(temp_h) - dt*temp_y*np.cos(temp_h)],
                               [dt*np.sin(temp_h), dt*np.cos(temp_h), 0, 1, dt*temp_x*np.cos(temp_h) - dt*temp_y*np.sin(temp_h)],
                               [1, dt*z[2][j], 0, 0, 0], 
                               [-dt*z[2][j], 1, 0, 0, 0],
                               [0, 0, 0, 0, 1]])

        # measurement covariance matrix
        ekfilter.R = z[3][j]

        # system error matrix
        # - initialized to zero matrix for first update
        ekfilter.Q = np.array([[0, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0]])

        # residual and kalman gain
        # - not computed for first update but initialized so it could be output
        residual = np.array([[0, 0],
                             [0, 0]])
        K = np.array([[0, 0],
                      [0, 0],
                      [0, 0],
                      [0, 0]])

    # Reinitialize State
    if updateNumber == 1: # Second Update

        prev_x = ekfilter.x[0][0]
        prev_y = ekfilter.x[1][0]
        prev_xv = ekfilter.x[2][0]
        prev_yv = ekfilter.x[3][0]
        prev_h = ekfilter.x[4][0]

        # x = R*lon*cos(lat0)
        temp_x = Radius*z[0][j]*np.cos(phi1)
        # y = R*lat
        temp_y = Radius*z[1][j]
        temp_xv = prev_xv + prev_yv*(z[2][j] - prev_h) +dt*z[4][j]*np.cos(z[2][j])
        temp_yv = prev_yv - prev_xv*(z[2][j] - prev_h) +dt*z[4][j]*np.sin(z[2][j])
        temp_h = z[2][j]

        # state vector
        # - reinitialized with new position and computed velocity
        ekfilter.x = np.array([[temp_x],
                               [temp_y],
                               [temp_xv],
                               [temp_yv],
                               [temp_h]])

        # state covariance matrix
        # - initialized to large value
        ekfilter.P = np.array([[100, 0, 0, 0, 0],
                               [0, 100, 0, 0, 0],
                               [0, 0, 200, 0, 0],
                               [0, 0, 0, 200, 0],
                               [0, 0, 0, 0, 100]])

        # state transistion matrix
        # - linear extrapolation
        ekfilter.A = np.array([[dt*np.cos(temp_h), dt*np.sin(temp_h), 1, 0, -dt*temp_x*np.sin(temp_h) - dt*temp_y*np.cos(temp_h)],
                               [dt*np.sin(temp_h), dt*np.cos(temp_h), 0, 1, dt*temp_x*np.cos(temp_h) - dt*temp_y*np.sin(temp_h)],
                               [1, dt*z[2][j], 0, 0, 0], 
                               [-dt*z[2][j], 1, 0, 0, 0],
                               [0, 0, 0, 0, 1]])

        # measurement covariance matrix
        # - provided by the measurment source
        ekfilter.R = z[3][j]

        # system error matrix
        # - initialized to an random value
        ekfilter.Q = np.array([[2, 0, 0, 0, 0],
                               [0, 2, 0, 0, 0],
                               [0, 0, 2, 0, 0],
                               [0, 0, 0, 2, 0],
                               [0, 0, 0, 0, 1]])
        # residual and kalman gain
        # - initialized so it could be output
        residual = np.array([[0, 0],
                      [0, 0]])
        K = np.array([[0, 0],
                      [0, 0],
                      [0, 0],
                      [0, 0]])
    
    if updateNumber > 1:

      # Predict State Forward (as linear)
      x_prime = ekfilter.A.dot(ekfilter.x)

      # Predict Covariance Forward (as linear)
      P_prime = ekfilter.A.dot(ekfilter.P).dot(ekfilter.A.T) + ekfilter.Q

      # state to measurement matrix (different in linear)
      ekfilter.H = np.array([[1/(Radius*np.cos(phi1)), 0, 0, 0, 0],
                             [0, 1/Radius, 0, 0, 0],
                             [0, 0, 0, 0, 1]])

      ekfilter.HT = np.array([[1/(Radius*np.cos(phi1)), 0, 0],
                              [0, 1/Radius, 0],
                              [0, 0, 0],
                              [0, 0, 0],
                              [0, 0, 1]])

      # measurement covariance matrix
      ekfilter.R = z[3][j]

      # Compute Kalman Gain (as linear)
      S = ekfilter.H.dot(P_prime).dot(ekfilter.HT) + ekfilter.R
      K = P_prime.dot(ekfilter.HT).dot(np.linalg.inv(S))

      # Estimate State (h is different wrt linear)
      # temp_z = current measurement in lon, lat and azimuth
      temp_z = np.array([[z[0][j]],
                         [z[1][j]],
                         [z[2][j]]])

      # compute the predicted range and azimuth (zp = h*xp)
      pred_x = x_prime[0][0]
      pred_y = x_prime[1][0]

      pred_lon = pred_x/(Radius*np.cos(phi1))
      pred_lat = pred_y/Radius
      pred_h = x_prime[4][0]
      h_small = np.array([[pred_lon],
                       [pred_lat],
                       [pred_h]])

      # compute the residual
      # - the difference between the state and measurement for that data time
      residual = temp_z - h_small

      # Compute new estimate for state vector using the Kalman Gain (as linear)
      ekfilter.x = x_prime + K.dot(residual)

      # Compute new estimate for state covariance using the Kalman Gain (as linear)
      ekfilter.P = P_prime - K.dot(ekfilter.H).dot(P_prime)

    return [ekfilter.x[0], ekfilter.x[1], ekfilter.x[2], ekfilter.x[3], ekfilter.x[4], ekfilter.P, K, residual, updateNumber];

#### EKF - Test

In [ ]:
def testFilter():
    measTime = []
    estPosx = []
    estVelx = []
    estPosy = []
    estVely = []
    estHeading = []

    z = getMeasurement()

    for iii in range(0,len(z[0])):
        # for each measurement, call the Extended Kalman Filter function
        f = ekfilter(z, iii)
        
        # Save off that state so that it could be plotted
        measTime.append(iii)
        estVelx.append(f[0])
        estVely.append(f[1])
        estPosx.append(f[2])
        estPosy.append(f[3])
        estHeading.append(f[4])
    
    return [estPosx, estPosy, estVelx, estVely, estHeading, z[0], z[1], z[2], measTime]; 

#### Plotting

In [ ]:
t = testFilter()
phi1 = 43
Radius = 371*10**3

measPosx = [ x * Radius* np.cos(phi1) for x in t[5] ]
measPosy = [ x * Radius for x in t[6] ]

plot1 = plt.figure(1)
plt.plot(t[8], t[5],)
plt.ylabel('Longitude with error')
plt.xlabel('Time')
plt.legend(['Measured'])
plt.grid(True)

plot2 = plt.figure(2)
plt.plot(t[8], t[6],)
plt.ylabel('Latitude with error')
plt.xlabel('Time')
plt.legend(['Measured'])
plt.grid(True)

plot3 = plt.figure(3)
plt.plot(t[8], t[2])
plt.ylabel('Velocity (meters/seconds)')
plt.xlabel('Update Number')
plt.title('Velocity Estimate On x direction \n', fontweight="bold")
plt.legend(['Estimate'])
plt.grid(True)

plot4 = plt.figure(4)
plt.plot(t[8], t[3])
plt.ylabel('Velocity (meters/seconds)')
plt.xlabel('Update Number')
plt.title('Velocity Estimate On y direction \n', fontweight="bold")
plt.legend(['Estimate'])
plt.grid(True)

plot5 = plt.figure(5)
plt.plot(t[8], t[4])
plt.ylabel('Heading')
plt.xlabel('Update Number')
plt.title('Heading Estimate', fontweight="bold")
plt.legend(['Estimate'])
plt.grid(True)

plot6 = plt.figure(6)
plt.scatter(t[8], t[0], color = 'red')
plt.plot(t[7], measPosx)
plt.ylabel('Position in x direction')
plt.xlabel('Time')
plt.legend(['Estimate','Measured'])
plt.grid(True)

plot7 = plt.figure(7)
plt.scatter(t[8], t[1], color = 'red')
plt.plot(t[8], measPosy)
plt.ylabel('Position in y direction')
plt.xlabel('Time')
plt.legend(['Estimate','Measured'])
plt.grid(True)

plot8 = plt.figure(8)
plt.scatter(t[0], t[1], color = 'red')
plt.plot(measPosx, measPosy)
plt.ylabel('Position')
plt.xlabel('Update Number')
plt.title('Estimated position in xy plane \n', fontweight="bold")
plt.legend(['Estimate', 'Measured'])
plt.grid(True)